In [ ]:
table_merge

In [ ]:
将其他几张表的有用特征加入到总表中

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_x = pd.read_csv('dataset/train/ord_train.csv')
train_del = ['confirmdate','orderstatus', 'price', 'commission','ordroomnum']
train_x.drop(train_del,axis = 1,inplace = True)

In [ ]:
母酒店维表

In [3]:
mhotel = pd.read_csv('dataset/train/mhotelinfo.csv')

In [30]:
mhotel.head()

,masterhotelid,glon,glat,star,zone,city,country
0,8657540,151.346340,-33.516495,0,NaN,4129,15
1,10095686,92.526480,58.190506,0,NaN,71743,30
2,15935739,34.558693,49.588090,0,NaN,5615,119
3,15156172,110.363140,-7.771237,2,0.0,741,108
4,10950378,20.406840,44.806694,0,2298.0,10257,183


In [31]:
set1 = set(mhotel['masterhotelid'])
set2 = set(train_x['masterhotelid'])

In [32]:
set3 = set2 - set1

In [34]:
len(list(set3))

3359

In [35]:
train_x.shape

(4023086, 20)

In [4]:
train_x = pd.merge(train_x,mhotel[['masterhotelid','glon','glat']],how = 'left')

In [37]:
train_x.shape

(4023086, 22)

In [38]:
temp = train_x[train_x['glon'].isnull()]
len(list(set(temp['masterhotelid'])))

3359

In [ ]:
至少上边没有问题了

In [ ]:
物理房型维表

In [5]:
mroom = pd.read_csv('dataset/train/mroominfo.csv')

In [40]:
mroom.head()

,masterbasicroomid,totalrooms,masterhotelid
0,10023629,1,3177583
1,10030510,3,2193611
2,10031578,2,735007
3,10032516,2,3035324
4,10034229,588,757098


In [6]:
train_x['masterbasicroomid'].fillna(-1,inplace = True)

In [7]:
train_x['masterbasicroomid'] = train_x['masterbasicroomid'].astype(int)

In [43]:
train_x.shape

(4023086, 22)

In [45]:
set1 = set(mroom['masterbasicroomid'])
set2 = set(train_x['masterbasicroomid'])

In [46]:
set3 = set2 - set1

In [47]:
len(list(set3))

5964

In [59]:
temp3 = mroom[['masterbasicroomid','totalrooms']].drop_duplicates()

In [60]:
temp3.shape

(21898029, 2)

In [61]:
mroom.shape

(21898126, 3)

In [ ]:
原因非常清楚了。必须结合另一个特征（母酒店），不然有些id是重复的。即不同的母酒店可能用了同一个物理房型的id
这样自然最后merge时会增加新的行。相当于对于同一个物理房型，它有两种totalroom值。
这必须用母酒店来区分。

In [8]:
train_x = pd.merge(train_x,mroom,on = ['masterbasicroomid','masterhotelid'],how = 'left')

In [9]:
train_x.shape

(4023086, 23)

In [11]:
train_x.rename(columns={'totalrooms':'wuli_totalrooms'}, inplace = True)

In [12]:
train_x.head()

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,...,masterhotelid,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,glon,glat,wuli_totalrooms
0,3678052984,2017-05-15 10:03:34,1777,111,6459206,3302.0,62786796,F,2017-06-23,2017-06-25,...,713478,1897.0,0,SHT,T,5,合作,109.212110,12.134152,430.0
1,3623712471,2017-05-02 21:25:37,622,4,4515821,NaN,51183791,F,2017-05-05,2017-05-06,...,1210465,2322.0,1,SHT,T,4,合作,100.891685,12.951805,6.0
2,3935430746,2017-06-20 15:03:39,366,30,3403530,NaN,18594989,F,2017-07-01,2017-07-04,...,741340,3686.0,1,SHT,F,5,合作,37.621204,55.758766,230.0
3,4095204178,2017-07-19 23:45:12,274,42,4984070,705.0,72423477,T,2017-08-22,2017-08-24,...,5033399,2299.0,0,HTL,T,5,直签,126.978410,37.558410,317.0
4,3753007353,2017-05-26 16:21:36,723,108,3796626,340.0,55656893,F,2017-05-27,2017-05-28,...,2144683,2391.0,0,SHT,T,5,合作,115.224790,-8.782472,692.0


In [ ]:
子酒店维表

In [13]:
hotel = pd.read_csv('dataset/train/hotelinfo.csv')

In [14]:
hotel.head()

,hotel,totalrooms,isebookinghtl,hotelbelongto,supplierid,masterhotelid
0,8164,2,F,PKG,NaN,8164
1,19660,4,F,HTL,NaN,19660
2,21524,2,F,HTL,NaN,21524
3,21653,1,F,HTL,2654.0,994188
4,22486,2,F,HTL,NaN,22486


In [15]:
temp3 = hotel[['hotel','totalrooms']].drop_duplicates()

In [16]:
temp3.shape,hotel.shape

((4629010, 2), (4629010, 6))

In [17]:
train_x = pd.merge(train_x,hotel[['hotel','totalrooms']],how = 'left')

In [18]:
train_x.shape

(4023086, 24)

In [19]:
train_x.rename(columns={'totalrooms':'zifangxing_totalrooms'}, inplace = True)

In [ ]:
先存档，之后再处理房态表

In [20]:
train_x.to_csv('dataset/train/ord_train_update.csv',index = False)

In [ ]:
读取方式：
train_x = pd.read_csv('dataset/train/ord_train_update.csv',encoding = 'GB2312')

In [ ]:
----房态表先不处理。实在是太复杂了。先用这些就够了

In [ ]:
然后是testAB的处理

In [8]:
test_A = pd.read_csv('dataset/test/ord_testA.csv',encoding = 'GB2312')
test_B = pd.read_csv('dataset/test/ord_testB.csv',encoding = 'GB2312')

In [ ]:
先是母酒店

In [3]:
test_A.head()

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,ordadvanceday,masterbasicroomid,masterhotelid,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel
0,4505896718,2017-09-15 14:11:00,633,66,6265739,-1.0,71340604,T,2017/10/11,2017/10/15,27,22012743,5633633,2299,0,HTL,T,3,直签
1,4504466510,2017-09-15 04:32:00,410,42,2811117,NaN,70578904,F,2017/9/19,2017/9/20,4,24779139,688942,3536,1,SHT,F,3,合作
2,4505791849,2017-09-15 13:43:00,301,111,8198646,853.0,91096440,F,2017/9/16,2017/9/17,2,50438281,7071803,2322,1,SHT,F,3,合作
3,4505284465,2017-09-15 11:38:00,274,42,3717320,705.0,20487676,F,2017/9/17,2017/9/20,2,11745244,991852,3965,1,SHT,F,0,合作
4,4506807614,2017-09-15 17:38:00,219,78,7217820,-1.0,88163053,T,2017/9/17,2017/9/20,2,38411936,7217821,2299,0,HTL,T,4,直签


In [4]:
mhotel = pd.read_csv('dataset/train/mhotelinfo.csv')

In [9]:
test_A.shape,test_B.shape

((11035, 19), (11036, 19))

In [10]:
test_A = pd.merge(test_A,mhotel[['masterhotelid','glon','glat']],how = 'left')
test_B = pd.merge(test_B,mhotel[['masterhotelid','glon','glat']],how = 'left')

In [11]:
test_A.shape,test_B.shape

((11035, 21), (11036, 21))

In [ ]:
物理房型

In [12]:
mroom = pd.read_csv('dataset/train/mroominfo.csv')

In [16]:
test_A = pd.merge(test_A,mroom,on = ['masterbasicroomid','masterhotelid'],how = 'left')
test_A.rename(columns={'totalrooms':'wuli_totalrooms'}, inplace = True)
test_B = pd.merge(test_B,mroom,on = ['masterbasicroomid','masterhotelid'],how = 'left')
test_B.rename(columns={'totalrooms':'wuli_totalrooms'}, inplace = True)

In [17]:
test_A.shape,test_B.shape

((11035, 22), (11036, 22))

In [ ]:
子酒店

In [18]:
hotel = pd.read_csv('dataset/train/hotelinfo.csv')

In [19]:
test_A = pd.merge(test_A,hotel[['hotel','totalrooms']],how = 'left')
test_A.rename(columns={'totalrooms':'zifangxing_totalrooms'}, inplace = True)
test_B = pd.merge(test_B,hotel[['hotel','totalrooms']],how = 'left')
test_B.rename(columns={'totalrooms':'zifangxing_totalrooms'}, inplace = True)

In [20]:
test_A.shape,test_B.shape

((11035, 23), (11036, 23))

In [21]:
test_A.to_csv('dataset/test/ord_testA_update.csv',index = False)
test_B.to_csv('dataset/test/ord_testB_update.csv',index = False)

In [ ]:
分析房态表

In [1]:
import numpy as np
import pandas as pd

In [4]:
train_x = pd.read_csv('dataset/train/ord_train_update.csv',encoding = 'GB2312')

In [5]:
train_x.head()

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,...,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,glon,glat,wuli_totalrooms,zifangxing_totalrooms
0,3678052984,2017-05-15 10:03:34,1777,111,6459206,3302.0,62786796,F,2017-06-23,2017-06-25,...,1897.0,0,SHT,T,5,合作,109.212110,12.134152,430.0,10.0
1,3623712471,2017-05-02 21:25:37,622,4,4515821,NaN,51183791,F,2017-05-05,2017-05-06,...,2322.0,1,SHT,T,4,合作,100.891685,12.951805,6.0,36.0
2,3935430746,2017-06-20 15:03:39,366,30,3403530,NaN,18594989,F,2017-07-01,2017-07-04,...,3686.0,1,SHT,F,5,合作,37.621204,55.758766,230.0,119.0
3,4095204178,2017-07-19 23:45:12,274,42,4984070,705.0,72423477,T,2017-08-22,2017-08-24,...,2299.0,0,HTL,T,5,直签,126.978410,37.558410,317.0,166.0
4,3753007353,2017-05-26 16:21:36,723,108,3796626,340.0,55656893,F,2017-05-27,2017-05-28,...,2391.0,0,SHT,T,5,合作,115.224790,-8.782472,692.0,4.0


In [6]:
train_x.shape

(4023086, 24)

In [18]:
bk = pd.read_csv('dataset/train/ord_bkroomstatus.csv')
zq = pd.read_csv('dataset/train/ord_zqroomstatus.csv')

In [19]:
noroom = train_x[train_x['noroom'] == 1]

In [21]:
noroom.sort_values(['orderdate'],ascending = False,inplace = True)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
noroom.shape

(205733, 24)

In [27]:
noroom.iloc[12000:12050,:]

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,...,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,glon,glat,wuli_totalrooms,zifangxing_totalrooms
3952733,4435685338,2017-09-03 09:34:54,219,78,6315055,856.0,55841689,F,2017-11-10,2017-11-12,...,2698.0,0,PKG,T,3,合作,135.497970,34.673775,332.0,16.0
431655,4435683070,2017-09-03 09:33:17,725,4,9547623,825.0,84693096,F,2017-09-03,2017-09-04,...,8604.0,0,SHT,T,4,合作,98.299350,7.904510,1880.0,4.0
3187716,4435682143,2017-09-03 09:33:05,359,4,2641950,382.0,36541616,F,2017-11-02,2017-11-04,...,2299.0,0,HTL,T,4,直签,100.529396,13.747979,470.0,73.0
241219,4435679802,2017-09-03 09:31:38,359,4,1259646,382.0,85121317,F,2017-09-03,2017-09-04,...,2295.0,1,HTL,F,5,Agoda,100.540270,13.739340,1203.0,1374.0
77087,4435679924,2017-09-03 09:31:28,623,4,8647489,-1.0,77425832,F,2017-09-28,2017-10-01,...,2299.0,0,HTL,T,3,直签,98.992320,18.788720,48.0,27.0
2186483,4435678263,2017-09-03 09:30:53,228,78,1882987,801.0,55658200,F,2017-09-04,2017-09-07,...,2301.0,0,SHT,T,4,合作,139.719650,35.729683,453.0,98.0
2409408,4435677666,2017-09-03 09:30:34,1225,2,5859050,558.0,56864326,F,2017-09-03,2017-09-04,...,2321.0,0,HTL,T,2,直签,99.726166,6.292498,84.0,83.0
1848510,4435677688,2017-09-03 09:30:24,360,78,6995563,NaN,61775871,F,2017-09-16,2017-09-17,...,6138.0,1,SHT,F,2,合作,136.884580,35.168420,725.0,254.0
1274759,4435675808,2017-09-03 09:29:38,1393,2,5816493,577.0,58016013,F,2017-09-10,2017-09-13,...,5201.0,0,SHT,T,3,合作,116.070910,5.978308,168.0,11.0
3226096,4435666460,2017-09-03 09:25:11,4252,78,8173034,-1.0,70967250,F,2017-09-17,2017-09-19,...,2299.0,0,HTL,T,3,直签,130.023270,33.195335,16.0,8.0


In [33]:
temp2 = zq[zq['room'] == 36541616]
temp2.sort_values(['createtime'],ascending = False,inplace = True)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
temp2[40:]

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
701412,36541616,2017-09-11,N,2017-09-03 20:23:15,6368299,2641950,2199501
139020,36541616,2017-09-22,N,2017-09-03 11:02:06,6368299,2641950,2199501
324287,36541616,2017-11-02,N,2017-09-03 09:53:37,6368299,2641950,2199501
1686021,36541616,2017-10-27,N,2017-09-03 09:45:49,6368299,2641950,2199501
588552,36541616,2017-10-28,N,2017-09-03 09:45:49,6368299,2641950,2199501
1516152,36541616,2017-10-24,N,2017-09-01 16:42:37,6368299,2641950,2199501
1269863,36541616,2017-09-02,N,2017-09-01 12:05:24,6368299,2641950,2199501
295335,36541616,2017-09-09,N,2017-09-01 12:05:24,6368299,2641950,2199501
1238649,36541616,2017-10-25,G,2017-08-30 16:40:07,6368299,2641950,2199501
1725080,36541616,2017-10-27,G,2017-08-30 16:40:07,6368299,2641950,2199501


In [8]:
bk.head()

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
0,49359351,2017-10-01,N,2017-08-24 03:54:17,44039493,4076704,1857089
1,13037426,2017-08-25,N,2017-08-05 14:34:38,7924733,229256,699276
2,8573004,2017-08-10,N,2017-08-07 18:25:40,7769214,538682,688203
3,13619122,2017-08-22,N,2017-08-05 09:44:34,5787108,785102,1074098
4,39616190,2017-08-13,N,2017-08-05 19:15:17,18215309,5114310,5086203


In [9]:
train_x[train_x['room'] == 49359351]

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,...,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,glon,glat,wuli_totalrooms,zifangxing_totalrooms
535600,4158469622,2017-07-28 20:31:23,3880,146,4076704,11936.0,49359351,F,2017-10-01,2017-10-05,...,2294.0,1,HTL,F,5,BOOKING,73.20892,5.83071,49.0,24.0
2115564,3405360250,2017-03-24 02:48:11,3880,146,4076704,11936.0,49359351,F,2017-03-30,2017-04-01,...,2294.0,1,HTL,F,5,BOOKING,73.20892,5.83071,49.0,24.0


In [11]:
temp = bk[bk['room'] == 49359351]
temp.sort_values(['createtime'],inplace = True)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
房态时间713到915
而且能看到，对于已经被booking的，基本上后期就每天N一次，表示这个房间被预定了。
（并没有考虑预定的日期距离现在还很远）

In [12]:
temp

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
33822,49359351,2017-10-01,N,2017-08-03 20:10:52,44039493,4076704,1857089
9677,49359351,2017-10-01,N,2017-08-03 20:18:45,44039493,4076704,1857089
6644,49359351,2017-10-01,N,2017-08-04 20:45:09,44039493,4076704,1857089
107650,49359351,2017-10-01,N,2017-08-05 01:43:27,44039493,4076704,1857089
54573,49359351,2017-10-01,N,2017-08-05 06:44:31,44039493,4076704,1857089
9436,49359351,2017-10-01,N,2017-08-05 11:36:17,44039493,4076704,1857089
40246,49359351,2017-10-01,N,2017-08-05 15:36:20,44039493,4076704,1857089
84895,49359351,2017-10-01,N,2017-08-05 20:33:00,44039493,4076704,1857089
4131,49359351,2017-10-01,N,2017-08-06 01:33:59,44039493,4076704,1857089
91195,49359351,2017-10-01,N,2017-08-06 06:27:00,44039493,4076704,1857089


In [17]:
train_x[train_x['room'] == 13037426]

,orderid,orderdate,city,countryid,hotel,zone,room,isholdroom,arrival,etd,...,supplierid,isvendor,hotelbelongto,isebookinghtl,hotelstar,supplierchannel,glon,glat,wuli_totalrooms,zifangxing_totalrooms
118870,4012903796,2017-07-05 17:17:20,1391,32,229256,0.0,13037426,F,2017-07-05,2017-07-06,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
566735,3690615230,2017-05-17 11:49:10,1391,32,229256,0.0,13037426,F,2017-05-22,2017-05-23,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
793641,3571699441,2017-04-22 14:35:36,1391,32,229256,0.0,13037426,F,2017-05-01,2017-05-02,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
829722,4033470138,2017-07-09 04:16:11,1391,32,229256,0.0,13037426,F,2017-07-09,2017-07-10,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
971399,3932273823,2017-06-19 20:15:34,1391,32,229256,0.0,13037426,F,2017-10-04,2017-10-06,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
1351081,3979057564,2017-06-29 10:51:56,1391,32,229256,0.0,13037426,F,2017-09-14,2017-09-19,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
2257079,3968442835,2017-06-27 10:32:10,1391,32,229256,0.0,13037426,F,2017-10-24,2017-10-28,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
2363045,3998804384,2017-07-02 18:45:53,1391,32,229256,0.0,13037426,F,2017-07-12,2017-07-15,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
2429858,3753301765,2017-05-26 17:19:36,1391,32,229256,0.0,13037426,F,2017-08-25,2017-08-29,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0
3909737,3958646587,2017-06-25 08:48:13,1391,32,229256,0.0,13037426,F,2017-08-18,2017-08-20,...,2294.0,1,HTL,F,5,BOOKING,121.91811,11.970395,155.0,34.0


In [14]:
temp = bk[bk['room'] == 13037426]
temp.sort_values(['createtime'],inplace = True)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
